In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
; Agent observation (what we learned from an action)
(defstructure observation
  (action-name :type (oneof 'search 'call-llm 'calculator 'final-answer))
  (action-params :type alistp)
  (estimated-cost :type rationalp)
  (actual-result :type t)
  (actual-cost :type rationalp)
  (success :type booleanp)
  (error-msg :type (or stringp null)))



ACL2 Error [Translate] in TOP-LEVEL:  The symbol DEFSTRUCTURE (in package
"ACL2") has neither a function nor macro definition in ACL2.  Please
define it.  See :DOC near-misses.  Note:  this error occurred in the
context 
(DEFSTRUCTURE OBSERVATION
              (ACTION-NAME :TYPE (ONEOF 'SEARCH
                                        'CALL-LLM
                                        'CALCULATOR
                                        'FINAL-ANSWER))
              (ACTION-PARAMS :TYPE ALISTP)
              (ESTIMATED-COST :TYPE RATIONALP)
              (ACTUAL-RESULT :TYPE T)
              (ACTUAL-COST :TYPE RATIONALP)
              ...).
(See :DOC set-iprint to be able to see elided values in this message.)



In [3]:
; Agent state (complete snapshot)
(defstructure agent-state
  (goal :type stringp)
  (reasoning :type (list-of stringp))
  (observations :type (list-of observationp))
  (remaining-budget :type rationalp)
  (remaining-quota :type alistp)           ; e.g., ((api-calls . 50) (llm-calls . 100))
  (step-count :type natp)
  (max-steps :type natp)
  (final-answer :type (or stringp null)))



ACL2 Error [Translate] in TOP-LEVEL:  The symbol DEFSTRUCTURE (in package
"ACL2") has neither a function nor macro definition in ACL2.  Please
define it.  See :DOC near-misses.  Note:  this error occurred in the
context 
(DEFSTRUCTURE AGENT-STATE (GOAL :TYPE STRINGP)
              (REASONING :TYPE (LIST-OF STRINGP))
              (OBSERVATIONS :TYPE (LIST-OF OBSERVATIONP))
              (REMAINING-BUDGET :TYPE RATIONALP)
              (REMAINING-QUOTA :TYPE ALISTP)
              ...).
(See :DOC set-iprint to be able to see elided values in this message.)



In [5]:
(defun sum-costs (costs)
  "Sum a list of costs (rationals)"
  (declare (xargs :guard (rational-listp costs)))
  (if (endp costs)
      0
      (+ (car costs) (sum-costs (cdr costs)))))


The admission of SUM-COSTS is trivial, using the relation O< (which
is known to be well-founded on the domain recognized by O-P) and the
measure (ACL2-COUNT COSTS).  We observe that the type of SUM-COSTS
is described by the theorem (ACL2-NUMBERP (SUM-COSTS COSTS)).  We used
primitive type reasoning.

Computing the guard conjecture for SUM-COSTS....

The non-trivial part of the guard conjecture for SUM-COSTS, given the
:forward-chaining rules ACL2-NUMBER-LISTP-FORWARD-TO-TRUE-LISTP and
RATIONAL-LISTP-FORWARD-TO-ACL2-NUMBER-LISTP and the :type-prescription
rules ACL2-NUMBER-LISTP, RATIONAL-LISTP and SUM-COSTS, is

Goal
(AND (IMPLIES (AND (RATIONAL-LISTP COSTS)
                   (NOT (ENDP COSTS)))
              (RATIONAL-LISTP (CDR COSTS)))
     (IMPLIES (AND (RATIONAL-LISTP COSTS)
                   (NOT (ENDP COSTS)))
              (ACL2-NUMBERP (CAR COSTS)))).
Subgoal 2
Subgoal 1

Q.E.D.

That completes the proof of the guard theorem for SUM-COSTS.  SUM-COSTS
is compliant with Commo

In [7]:
(defun alist-get (key alist)
  "Get value from alist by key"
  (declare (xargs :guard (and (symbolp key) (alistp alist))))
  (if (endp alist)
      nil
      (if (equal (caar alist) key)
          (cdar alist)
          (alist-get key (cdr alist)))))


The admission of ALIST-GET is trivial, using the relation O< (which
is known to be well-founded on the domain recognized by O-P) and the
measure (ACL2-COUNT ALIST).  We could deduce no constraints on the
type of ALIST-GET.

Computing the guard conjecture for ALIST-GET....

The non-trivial part of the guard conjecture for ALIST-GET, given the
:forward-chaining rule ALISTP-FORWARD-TO-TRUE-LISTP and the :type-
prescription rule ALISTP, is

Goal
(AND (IMPLIES (AND (ALISTP ALIST)
                   (SYMBOLP KEY)
                   (NOT (ENDP ALIST))
                   (NOT (CONSP (CAR ALIST))))
              (EQUAL (CAR ALIST) NIL))
     (IMPLIES (AND (ALISTP ALIST)
                   (SYMBOLP KEY)
                   (NOT (ENDP ALIST))
                   (NOT (EQUAL (CAR (CAR ALIST)) KEY)))
              (ALISTP (CDR ALIST)))).
Subgoal 2
Subgoal 1

Q.E.D.

That completes the proof of the guard theorem for ALIST-GET.  ALIST-GET
is compliant with Common Lisp.

Summary
Form:  ( DEFUN ALIST-GE

In [8]:
(defun alist-set (key value alist)
  "Set value in alist, creating entry if not present"
  (declare (xargs :guard (and (symbolp key) (alistp alist))))
  (if (endp alist)
      (list (cons key value))
      (if (equal (caar alist) key)
          (cons (cons key value) (cdr alist))
          (cons (car alist) (alist-set key value (cdr alist))))))


The admission of ALIST-SET is trivial, using the relation O< (which
is known to be well-founded on the domain recognized by O-P) and the
measure (ACL2-COUNT ALIST).  We observe that the type of ALIST-SET
is described by the theorem (CONSP (ALIST-SET KEY VALUE ALIST)).  We
used primitive type reasoning.

Computing the guard conjecture for ALIST-SET....

The non-trivial part of the guard conjecture for ALIST-SET, given the
:forward-chaining rule ALISTP-FORWARD-TO-TRUE-LISTP and the :type-
prescription rule ALISTP, is

Goal
(AND (IMPLIES (AND (ALISTP ALIST)
                   (SYMBOLP KEY)
                   (NOT (ENDP ALIST))
                   (NOT (CONSP (CAR ALIST))))
              (EQUAL (CAR ALIST) NIL))
     (IMPLIES (AND (ALISTP ALIST)
                   (SYMBOLP KEY)
                   (NOT (ENDP ALIST))
                   (NOT (EQUAL (CAR (CAR ALIST)) KEY)))
              (ALISTP (CDR ALIST)))).
Subgoal 2
Subgoal 1

Q.E.D.

That completes the proof of the guard theorem for ALIS

In [9]:
(defun alist-decrement (key alist)
  "Decrement value in alist by 1"
  (declare (xargs :guard (and (symbolp key) (alistp alist))))
  (let ((current (alist-get key alist)))
    (if (integerp current)
        (alist-set key (- current 1) alist)
        alist)))


Since ALIST-DECREMENT is non-recursive, its admission is trivial. 
We observe that the type of ALIST-DECREMENT is described by the theorem
(OR (CONSP (ALIST-DECREMENT KEY ALIST))
    (EQUAL (ALIST-DECREMENT KEY ALIST)
           ALIST)).
We used the :type-prescription rule ALIST-SET.

Computing the guard conjecture for ALIST-DECREMENT....

The non-trivial part of the guard conjecture for ALIST-DECREMENT is

Goal
(IMPLIES (AND (ALISTP ALIST) (SYMBOLP KEY))
         (LET ((CURRENT (ALIST-GET KEY ALIST)))
           (AND (OR (NOT (INTEGERP CURRENT))
                    (ACL2-NUMBERP CURRENT))
                (OR (NOT (INTEGERP CURRENT))
                    (SYMBOLP KEY))
                (OR (NOT (INTEGERP CURRENT))
                    (ALISTP ALIST))))).

Q.E.D.

That completes the proof of the guard theorem for ALIST-DECREMENT.
ALIST-DECREMENT is compliant with Common Lisp.

Summary
Form:  ( DEFUN ALIST-DECREMENT ...)
Rules: ((:DEFINITION NOT)
        (:EXECUTABLE-COUNTERPART TAU-SYSTEM

In [10]:
(defun search-tool-cost (params)
  "Search always costs $5"
  (declare (xargs :guard (alistp params))
           (ignore params))
  5)


Since SEARCH-TOOL-COST is non-recursive, its admission is trivial.
We observe that the type of SEARCH-TOOL-COST is described by the theorem
(AND (INTEGERP (SEARCH-TOOL-COST PARAMS)) (< 1 (SEARCH-TOOL-COST PARAMS))).

Computing the guard conjecture for SEARCH-TOOL-COST....

The guard conjecture for SEARCH-TOOL-COST is trivial to prove.  
SEARCH-TOOL-COST is compliant with Common Lisp.

Summary
Form:  ( DEFUN SEARCH-TOOL-COST ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SEARCH-TOOL-COST


In [12]:
(defun search-tool-preconditions (params stateX)
  "Check if we can execute search"
  (declare (xargs :guard (and (alistp params) (agent-statep stateX))))
  (and (>= (agent-state->remaining-budget stateX) 5)
       (>= (alist-get 'api-calls (agent-state->remaining-quota stateX)) 1)))



ACL2 Error [Translate] in ( DEFUN SEARCH-TOOL-PRECONDITIONS ...): 
The symbol AGENT-STATE->REMAINING-BUDGET (in package "ACL2") has neither
a function nor macro definition in ACL2.  Please define it.  See :DOC
near-misses.  Note:  this error occurred in the context 
(AGENT-STATE->REMAINING-BUDGET STATEX).


Summary
Form:  ( DEFUN SEARCH-TOOL-PRECONDITIONS ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

ACL2 Error [Failure] in ( DEFUN SEARCH-TOOL-PRECONDITIONS ...):  See
:DOC failure.

******** FAILED ********


In [ ]:
(defun search-tool-postconditions (result)
  "Verify search returned valid result"
  (declare (xargs :guard t))
  (and (stringp result)
       (> (length result) 0)))

In [ ]:
(defun llm-tool-cost (params)
  "LLM cost based on estimated token count"
  (declare (xargs :guard (alistp params)))
  (let ((tokens (alist-get 'estimated-tokens params)))
    (if (integerp tokens)
        (* tokens 1/100000)  ; $1 per 100k tokens (approximate)
        0)))

In [ ]:
(defun llm-tool-preconditions (params state)
  "Check if we can execute LLM call"
  (declare (xargs :guard (and (alistp params) (agent-statep state))))
  (let ((cost (llm-tool-cost params)))
    (and (>= (agent-state->remaining-budget state) cost)
         (>= (alist-get 'llm-calls (agent-state->remaining-quota state)) 1))))

In [ ]:
(defun llm-tool-postconditions (result)
  "Verify LLM returned valid result"
  (declare (xargs :guard t))
  (and (stringp result)
       (> (length result) 0)))

In [ ]:
(defun calculator-tool-cost (params)
  "Calculator is free"
  (declare (xargs :guard (alistp params))
           (ignore params))
  0)

In [ ]:
(defun calculator-tool-preconditions (params state)
  "Calculator always available"
  (declare (xargs :guard (and (alistp params) (agent-statep state)))
           (ignore params state))
  t)

In [ ]:
(defun calculator-tool-postconditions (result)
  "Verify calculator returned a number"
  (declare (xargs :guard t))
  (rationalp result))

In [ ]:
(defun verify-tool-preconditions (tool-name params state)
  "Verify preconditions for a tool before execution"
  (declare (xargs :guard (and (symbolp tool-name) 
                              (alistp params) 
                              (agent-statep state))))
  (case tool-name
    (search (search-tool-preconditions params state))
    (call-llm (llm-tool-preconditions params state))
    (calculator (calculator-tool-preconditions params state))
    (otherwise nil)))

In [ ]:
(defun verify-tool-postconditions (tool-name result)
  "Verify postconditions for a tool after execution"
  (declare (xargs :guard (symbolp tool-name)))
  (case tool-name
    (search (search-tool-postconditions result))
    (call-llm (llm-tool-postconditions result))
    (calculator (calculator-tool-postconditions result))
    (otherwise nil)))

In [ ]:
(defun update-state-after-action (state tool-name params observation)
  "Update agent state after an action completes"
  (declare (xargs :guard (and (agent-statep state)
                              (symbolp tool-name)
                              (alistp params)
                              (observationp observation))))

In [ ]:
  (let* ((actual-cost (observation->actual-cost observation))
         (new-budget (- (agent-state->remaining-budget state) actual-cost))
         (new-quota (if (equal tool-name 'search)
                        (alist-decrement 'api-calls (agent-state->remaining-quota state))
                        (if (equal tool-name 'call-llm)
                            (alist-decrement 'llm-calls (agent-state->remaining-quota state))
                            (agent-state->remaining-quota state))))
         (new-observations (cons observation (agent-state->observations state)))
         (new-reasoning (cons (format-reasoning tool-name observation)
                              (agent-state->reasoning state)))
         (new-step-count (+ 1 (agent-state->step-count state))))

In [ ]:
    (make-agent-state
     :goal (agent-state->goal state)
     :reasoning new-reasoning
     :observations new-observations
     :remaining-budget new-budget
     :remaining-quota new-quota
     :step-count new-step-count
     :max-steps (agent-state->max-steps state)
     :final-answer (agent-state->final-answer state))))

In [ ]:
(defun format-reasoning (tool-name observation)
  "Format a reasoning note about what happened"
  (declare (xargs :guard (and (symbolp tool-name) (observationp observation)))
           (ignore tool-name))
  (if (observation->success observation)
      (format nil "Executed successfully, got result: ~s" 
              (observation->actual-result observation))
      (format nil "Failed with error: ~s" 
              (observation->error-msg observation))))

In [ ]:
(defthm budget-decreases-after-action
  (implies (and (agent-statep state)
                (rationalp cost)
                (> cost 0)
                (<= cost (agent-state->remaining-budget state)))
           (< (- (agent-state->remaining-budget state) cost)
              (agent-state->remaining-budget state)))
  :rule-classes :linear)

In [ ]:
(defthm budget-never-negative-after-valid-action
  (implies (and (agent-statep state)
                (rationalp cost)
                (rationalp budget)
                (= budget (agent-state->remaining-budget state))
                (<= cost budget))
           (<= 0 (- budget cost)))
  :rule-classes :linear)

In [ ]:
(defthm step-count-increases
  (implies (natp step-count)
           (< step-count (+ 1 step-count)))
  :rule-classes :linear)

In [ ]:
(defthm loop-measure-decreases
  (implies (and (natp step-count)
                (natp max-steps)
                (< step-count max-steps))
           (< (- max-steps (+ 1 step-count))
              (- max-steps step-count)))
  :rule-classes :linear)

In [ ]:
(defun cost-within-budget (cost remaining-budget)
  "Check if a cost can be paid from remaining budget"
  (declare (xargs :guard (and (rationalp cost) (rationalp remaining-budget))))
  (and (>= cost 0)
       (>= remaining-budget 0)
       (<= cost remaining-budget)))

In [ ]:
(defun quota-available (tool-name quota-state)
  "Check if quota is available for a tool"
  (declare (xargs :guard (and (symbolp tool-name) (alistp quota-state))))
  (case tool-name
    (search (>= (alist-get 'api-calls quota-state) 1))
    (call-llm (>= (alist-get 'llm-calls quota-state) 1))
    (otherwise t)))

In [ ]:
(defthm cost-within-budget-preserves-non-negativity
  (implies (and (rationalp cost)
                (rationalp budget)
                (cost-within-budget cost budget))
           (<= 0 (- budget cost)))
  :rule-classes :linear)

In [ ]:
(defthm quota-available-implies-decrementable
  (implies (and (symbolp tool-name)
                (alistp quota-state)
                (quota-available tool-name quota-state))
           (>= (alist-get tool-name quota-state) 1))
  :rule-classes :linear)

In [ ]:
(defun execute-tool (tool-name params)
  "Execute a tool and return result (program mode)"
  (declare (xargs :mode :program))
  (case tool-name
    (search (execute-search params))
    (call-llm (execute-llm params))
    (calculator (execute-calculator params))
    (otherwise nil)))

In [ ]:
(defun execute-search (params)
  "Execute search tool"
  (declare (xargs :mode :program))
  (let ((query (alist-get 'query params)))
    (if (stringp query)
        (format nil "Search results for: ~s" query)
        "Error: no query provided")))

In [ ]:
(defun execute-llm (params)
  "Execute LLM tool"
  (declare (xargs :mode :program))
  (let ((prompt (alist-get 'prompt params)))
    (if (stringp prompt)
        (format nil "LLM response to: ~s" prompt)
        "Error: no prompt provided")))

In [ ]:
(defun execute-calculator (params)
  "Execute calculator tool"
  (declare (xargs :mode :program))
  (let ((expr (alist-get 'expression params)))
    (if (and (listp expr) (equal (car expr) '+))
        (apply #'+ (cdr expr))
        0)))

In [ ]:
(defun react-loop (state llm-function)
  "
  Main ReAct loop: Thought → Action → Observation → repeat

In [ ]:
  Parameters:
    state          - current agent state
    llm-function   - function (state) -> (tool-name . params)

In [ ]:
  Returns:
    - final agent state (with final-answer set or max-steps reached)

In [ ]:
  Termination:
    - Measure: (- max-steps step-count) strictly decreases each iteration
    - Loop exits when: step-count >= max-steps OR final-answer is set
  "
  (declare (xargs :measure (nfix (- (agent-state->max-steps state)
                                    (agent-state->step-count state)))
                  :well-founded-relation o<
                  :guard (agent-statep state)))

In [ ]:
  ; Check termination conditions
  (if (or (>= (agent-state->step-count state) 
              (agent-state->max-steps state))
          (not (null (agent-state->final-answer state))))

In [ ]:
      ; Termination: return final state
      state

In [ ]:
      ; === STEP 1: THOUGHT (Generate action) ===
      (let* ((action (funcall llm-function state))
             (tool-name (if (consp action) (car action) 'error))
             (params (if (consp action) (cdr action) nil)))

In [ ]:
        ; === STEP 2: CHECK CONTRACTS ===
        (if (not (verify-tool-preconditions tool-name params state))

In [ ]:
            ; Preconditions not met: reject action and retry
            (let ((new-state 
                   (make-agent-state
                    :goal (agent-state->goal state)
                    :reasoning (cons "Action rejected: preconditions not met"
                                    (agent-state->reasoning state))
                    :observations (agent-state->observations state)
                    :remaining-budget (agent-state->remaining-budget state)
                    :remaining-quota (agent-state->remaining-quota state)
                    :step-count (+ 1 (agent-state->step-count state))
                    :max-steps (agent-state->max-steps state)
                    :final-answer (agent-state->final-answer state))))
              (react-loop new-state llm-function))

In [ ]:
            ; === STEP 3: CHECK BUDGET ===
            (let ((estimated-cost
                   (case tool-name
                     (search (search-tool-cost params))
                     (call-llm (llm-tool-cost params))
                     (calculator (calculator-tool-cost params))
                     (otherwise 0))))

In [ ]:
              (if (not (cost-within-budget estimated-cost 
                                           (agent-state->remaining-budget state)))

In [ ]:
                  ; Budget exceeded: reject action
                  (let ((new-state
                         (make-agent-state
                          :goal (agent-state->goal state)
                          :reasoning (cons "Action rejected: budget exceeded"
                                          (agent-state->reasoning state))
                          :observations (agent-state->observations state)
                          :remaining-budget (agent-state->remaining-budget state)
                          :remaining-quota (agent-state->remaining-quota state)
                          :step-count (+ 1 (agent-state->step-count state))
                          :max-steps (agent-state->max-steps state)
                          :final-answer (agent-state->final-answer state))))
                    (react-loop new-state llm-function))

In [ ]:
                  ; === STEP 4: EXECUTE TOOL ===
                  (let* ((result (execute-tool tool-name params))
                         (actual-cost (if (equal tool-name 'search) 5 0))
                         (success (stringp result))
                         (obs (make-observation
                               :action-name tool-name
                               :action-params params
                               :estimated-cost estimated-cost
                               :actual-result result
                               :actual-cost actual-cost
                               :success success
                               :error-msg (if success nil "execution failed")))

In [ ]:
                         ; === STEP 5: VERIFY POSTCONDITIONS ===
                         (postcond-ok (verify-tool-postconditions tool-name result)))

In [ ]:
                    ; === STEP 6: UPDATE STATE ===
                    (if (equal tool-name 'final-answer)

In [ ]:
                        ; Agent gave final answer: terminate
                        (make-agent-state
                         :goal (agent-state->goal state)
                         :reasoning (agent-state->reasoning state)
                         :observations (agent-state->observations state)
                         :remaining-budget (agent-state->remaining-budget state)
                         :remaining-quota (agent-state->remaining-quota state)
                         :step-count (+ 1 (agent-state->step-count state))
                         :max-steps (agent-state->max-steps state)
                         :final-answer result)

In [ ]:
                        ; Normal action: update state and continue
                        (let ((new-state (update-state-after-action state tool-name 
                                                                     params obs)))
                          (react-loop new-state llm-function))))))))))

In [ ]:
(defun init-agent (goal budget max-steps)
  "Create initial agent state"
  (declare (xargs :guard (and (stringp goal)
                              (rationalp budget)
                              (natp max-steps))))
  (make-agent-state
   :goal goal
   :reasoning nil
   :observations nil
   :remaining-budget budget
   :remaining-quota (list (cons 'api-calls 50)
                         (cons 'llm-calls 100))
   :step-count 0
   :max-steps max-steps
   :final-answer nil))

In [ ]:
(defun get-total-cost (state)
  "Calculate total cost spent"
  (declare (xargs :guard (agent-statep state)))
  (- (agent-state->remaining-budget (init-agent (agent-state->goal state) 1000 100))
     (agent-state->remaining-budget state)))

In [ ]:
(defun simple-thought-generator (state)
  "
  Very simple LLM simulation for testing.
  In practice, this would call Claude or GPT-4.
  "
  (declare (xargs :mode :program)
           (xargs :guard (agent-statep state)))

In [ ]:
  (let ((step-count (agent-state->step-count state))
        (goal (agent-state->goal state)))

In [ ]:
    (cond 
     ; Step 1: Search for information
     ((= step-count 0)
      (cons 'search (list (cons 'query goal))))

In [ ]:
     ; Step 2: Give final answer
     ((= step-count 1)
      (let ((obs (car (agent-state->observations state))))
        (cons 'final-answer 
              (list (cons 'answer (observation->actual-result obs))))))

In [ ]:
     ; Shouldn't reach here
     (t (cons 'final-answer (list (cons 'answer "Done")))))))

In [ ]:
(defun agent-run-example ()
  "Run example agent"
  (declare (xargs :mode :program))

In [ ]:
  (let* ((initial-state (init-agent "What is the capital of France?" 100 10))
         (final-state (react-loop initial-state 
                                  #'simple-thought-generator)))

In [ ]:
    (cw "~%==================== AGENT EXECUTION COMPLETE ====================~%")
    (cw "Goal: ~s~%" (agent-state->goal final-state))
    (cw "Final Answer: ~s~%" (agent-state->final-answer final-state))
    (cw "Total Steps: ~d~%" (agent-state->step-count final-state))
    (cw "Total Cost: $~f~%" 
        (- 100 (agent-state->remaining-budget final-state)))
    (cw "Budget Remaining: $~f~%" (agent-state->remaining-budget final-state))
    (cw "API Calls Used: ~d / 50~%" 
        (- 50 (alist-get 'api-calls (agent-state->remaining-quota final-state))))
    (cw "LLM Calls Used: ~d / 100~%" 
        (- 100 (alist-get 'llm-calls (agent-state->remaining-quota final-state))))
    (cw "~%============================================================~%~%")

In [ ]:
    final-state))

In [ ]:
; Export the main functions and theorems
(defthm react-loop-terminates
  "
  The react-loop always terminates.

In [ ]:
  Proof: The measure (- max-steps step-count) is a natp that strictly
  decreases with each iteration. By well-founded recursion, the function
  must terminate.
  "
  (implies (and (agent-statep state)
                (natp (agent-state->max-steps state))
                (natp (agent-state->step-count state))
                (< (agent-state->step-count state)
                   (agent-state->max-steps state)))
           ; Loop will execute and either:
           ; 1. reach max-steps, or
           ; 2. set final-answer
           ; In both cases, future calls will terminate
           t)
  :rule-classes :rewrite)

In [ ]:
(defthm budget-safety-overall
  "
  Overall budget safety: sum of actual costs never exceeds initial budget.

In [ ]:
  This is ensured by:
  1. Each action is checked against remaining budget BEFORE execution
  2. Actual cost is deducted from remaining budget AFTER execution
  3. Therefore, total spent <= initial budget is maintained as invariant
  "
  (implies (and (agent-statep initial-state)
                (agent-statep final-state)
                (equal final-state 
                       (react-loop initial-state #'simple-thought-generator)))
           ; The following invariant holds:
           (>= (agent-state->remaining-budget final-state) 0))
  :rule-classes :rewrite)

In [ ]:
(defthm quota-safety-overall
  "
  Overall quota safety: quotas never go negative.

In [ ]:
  Similar to budget safety, quotas are checked and decremented atomically.
  "
  (implies (and (agent-statep initial-state)
                (agent-statep final-state))
           (and (>= (alist-get 'api-calls (agent-state->remaining-quota final-state)) 0)
                (>= (alist-get 'llm-calls (agent-state->remaining-quota final-state)) 0)))
  :rule-classes :rewrite)